In [273]:
import json
from datetime import datetime

fines = []

fines_json = json.loads(json.loads(open('fines.json').read())['Value'])['Fines']

for fine in fines_json:
    fines.append({
        'Дата и время': datetime.strptime(fine['ApnDetail'][2]['Value'], '%d.%m.%Y %H:%M:%S'),
        'Номер постановления': fine['DAP'],
        'Статья КоАП': fine['ApnDetail'][0]['Value'].replace('\t', ' - '),
        'Место правонарушения': fine['ApnDetail'][3]['Value'],
        'Lat': fine['Latitude'],
        'Lng': fine['Longitude'],
        'Штраф': fine['FineSum'],
    })

In [274]:
#fines

In [275]:
import pandas as pd

fines_df = pd.DataFrame.from_dict(fines)

print(fines_df.shape)
fines_df.head()

(835, 7)


,Lat,Lng,Дата и время,Место правонарушения,Номер постановления,Статья КоАП,Штраф
0,"55,7563209999","37,614136",2018-01-23 00:21:00,МОСКВА Г. МОХОВАЯ д.15/1,18810177180125209293,12.17 ч.1.2 - Движение или остановка на полосе...,3000
1,"55,7911569999","37,260069",2017-09-10 10:31:00,"А/Д М-9 ""БАЛТИЯ"", 24КМ+750М, ИЗ МОСКВЫ",18810150170911043896,12.09.2 - Превышение скорости движения ТС от 2...,500
2,"55,7391939999","37,433588",2017-07-24 10:19:00,МОСКВА Г. РУБЛЕВСКОЕ Ш. д.79,18810177170728394158,12.9.6 - Повторное превышение скорости на 40-6...,2000
3,"55,732458","37,496129",2017-06-20 00:07:00,МОСКВА Г. КУТУЗОВСКИЙ ПРОСП.,18810177170621061208,12.15.1 - Наруш.правил расп.т/с на пр.части бе...,1500
4,"55,752697","37,577857",2017-06-22 19:47:00,МОСКВА Г. НОВЫЙ АРБАТ д.36/9,18810177170702113941,12.9.6 - Повторное превышение скорости на 40-6...,2000


In [276]:
summary = fines_df.groupby('Статья КоАП')['Штраф'].agg(['count', 'sum']).sort_values(['count'], ascending=False).rename(index=str, columns={"count": "Количество штрафов", "sum": "Сумма штрафов"})
days_count = (fines_df['Дата и время'].max() - fines_df['Дата и время'].min()).days
summary.append(pd.Series(summary.sum(),name='Итого за {} дней'.format(days_count)))

,Количество штрафов,Сумма штрафов
Статья КоАП,,
12.09.2 - Превышение скорости движения ТС от 20 до 40 км/ч,361,180500
12.17 ч.1.2 - Движение или остановка на полосе для маршрутных ТС в Москве или в Санкт-Петербурге,212,636000
"12.15.1 - Наруш.правил расп.т/с на пр.части без выезда на ст.встр.движ.,дв.по обоч.,пересечение колонны",95,142500
12.9.6 - Повторное превышение скорости на 40-60 км/час,94,188000
"12.15.5 - Повторный выезд на сторону, предназначенную для встречного движения",21,105000
12.9.7 - Повторное превышение скорости на 60-80 км/час и более 80 км/ч,17,85000
"12.16.5 Нарушение, предусмотренное частью 4 настоящей статьи, совершенное в городе федерального значения Москве или Санкт-Петербурге",15,45000
"12.16.1 - Несоблюдение требований знаков или разметки, за искл.случаев, предусм.др.статьями гл.12",8,4000
12.09.3 - Превышение скорости движения ТС от 40 до 60 км/ч,7,7000
